In [1]:
#default_exp train3d

In [2]:
#export
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(5)

In [4]:
df_any = Meta.df_any
df_any.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_66891ac22,CT,ID_42940b2c,ID_17e33f43d0,ID_e14dd0090b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
ID_8e6e5b51f,CT,ID_76fbed32,ID_1d8eaa14ef,ID_e3919709a0,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,0,0,0,1
ID_cb8b9b514,CT,ID_20039b63,ID_1cfe3e70dd,ID_e3b5d8d9b8,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_ee683911f,CT,ID_a065f3ac,ID_9c727ac231,ID_99e83a310d,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0


In [5]:
# df_slice_count = df_any.groupby(['SeriesInstanceUID']).agg(['count'])
# max(df_slice_count.PatientID.values)

# min(df_slice_count.PatientID.values)

# max_seq_len = 60

In [6]:
#export
class OpenCTs:
    def __init__(self, path): 
        self.fn = get_pil_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        xs
        return TensorCTScan(torch.stack(xs))

In [7]:
#export
def pad_batch(x, pad_to=None, value=0):
    if isinstance(x, tuple): return tuple([pad_batch(s, pad_to, value) for s in x])
    bs_pad = pad_to-x.shape[0]
    pad = [0]*len(x.shape)*2
    pad[-1] = bs_pad
    return F.pad(x, pad=pad, value=value)

def pad_collate(items, values=[0,-1], fixed_pad=None):
    def get_bs(x): return x[0].shape[0] if hasattr(x[0], 'shape') else get_bs(x[0])
#     pad_to = max([get_bs(x) for x in items]) if fixed_pad is None else fixed_pad
    pad_to = 60
    def pad_row(row, pad_to, vals): return tuple([pad_batch(x,pad_to,v) for x,v in zip(row,vals)])
    return [pad_row(row, pad_to, values) for row in items]

In [8]:
#export
class TfmSOP:
    def __init__(self,df,open_fn):
        self.open_fn = open_fn
        self.df = df
    
    def x(self, sid):
        sids = self.df.SOPInstanceUID[sid].values
        return self.open_fn(sids)
    
    def y(self, sid): 
        vals = self.df.loc[sid,htypes].values
        return TensorMultiCategory(tensor(vals)).float()

In [9]:
#export
def get_3d_dsrc(df, open_fn, grps=Meta.grps, cv_idx=0, tfms=None, column='SeriesInstanceUID', test=False):
    df_series = df.reset_index().set_index(column).sort_values([column, "ImagePositionPatient2"])
    sids = df_series.index.unique()
    sid2idx = dict(zip(sids, range(len(sids))))
    
    # multi index is 10x faster
    df_series.index = pd.MultiIndex.from_tuples(df_series.index.str.split('|').tolist())
    tfm = TfmSOP(df_series, open_fn)
    
    if test: return DataSource(sids, [[tfm.x]], splits=[L.range(sids), L.range(sids)])
    
    s1 = [sid2idx[sid] for sid in group_cv(cv_idx,grps) if sid in sid2idx]
    s2 = [sid2idx[sid] for sid in grps[cv_idx] if sid in sid2idx]
    dsrc = DataSource(sids, [[tfm.x]+L(tfms),[tfm.y]], splits=(s1,s2))
    return dsrc

In [10]:
#export
def get_dbunch(dsrc, bs, batch_tfms, num_workers=8):
    dbunch = DataBunch(
        TfmdDL(dsrc.train, bs=bs, before_batch=[pad_collate], after_batch=batch_tfms, num_workers=num_workers, shuffle=True),
        TfmdDL(dsrc.valid, bs=bs, before_batch=[pad_collate], after_batch=batch_tfms, num_workers=num_workers)
    )
    dbunch.device = default_device()
    dbunch.c = 6
    return dbunch

In [11]:
#export
def get_3d_dbunch(df, path=path_jpg256, bs=1, num_workers=8, tfms=None):
    dsrc = get_3d_dsrc(df, open_fn=OpenCTs(path))

    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(nrm, Cuda(), IntToFloatTensor(), *L(tfms))
    
    return get_dbunch(dsrc, bs, batch_tfms, num_workers)


In [12]:
#export
def get_3d_dbunch_aug(df, path=path_jpg256, bs=1, num_workers=8):
    nrm = Normalize.from_stats(mean,std)
    batch_tfms = L(nrm, Cuda())
    dsrc = get_3d_dsrc(df, open_fn=OpenCTs(path), tfms=[IntToFloatTensor()]+aug_transforms())
    
    return get_dbunch(dsrc, bs, batch_tfms, num_workers)


In [16]:
# dbunch = get_3d_dbunch_aug(df_any, bs=10)
# next(iter(dbunch.train_dl))

In [17]:
#export
class Wrap():
    def __init__(self, tfm, tfm_all=True): self.tfm = tfm
    def __getattr__(self, x): return getattr(self.tfm, x)
    def __call__(self, *args, **kwargs): return self.encodes(*args, **kwargs)
    def encodes(self, x:TensorImage): return self.reshape(x, self.tfm)
    def decodes(self, x:TensorImage): return self.reshape(x, self.tfm.decodes)
    
    def reshape(self, x, func):
        if len(x.shape) != 5: return x
        bs,ts,ch,w,h = x.shape
        x = x.reshape(-1,ch,w,h)
        out = func(x)
        return out.reshape(bs,ts, *out.shape[-3:])
        

# wrapped_tfms = [Wrap(tfm) for tfm in aug_transforms()]

In [18]:
# dsrc = get_3d_dsrc(df_any, open_fn=OpenCTs(path_jpg256))
# x,y = dsrc[0]
# x.shape, y.shape

# dbunch = get_3d_dbunch(df_any, bs=10)
# x,y = dbunch.one_batch()
# x.shape, y.shape

## Features

In [19]:
#export
def get_np_fn(p):
    def _f(fn): return torch.from_numpy(np.load(str(p/f'{fn}.npy')))
    return _f

In [20]:
#export
class OpenFeats:
    def __init__(self, path):
        self.fn = get_np_fn(path)
        self.tt = ToTensor()
    def __call__(self, item):
        if isinstance(item, (str, Path)): return self.fn(item)
        xs = [self.tt(self.fn(x)) for x in item]
        return TensorCTScan(torch.stack(xs))

In [21]:
#export
def get_3d_dbunch_feat(df, path=path/'features_256', bs=1, num_workers=8):
    dsrc = get_3d_dsrc(df, open_fn=OpenFeats(path))
    return get_dbunch(dsrc, bs, [Cuda()], num_workers)


In [22]:
# dbunch_feat = get_3d_dbunch_feat(df_any, bs=10)
# xb,yb = dbunch_feat.one_batch()
# xb.shape, yb.shape

## Model

In [23]:
#export
class ReshapeBodyHook():
    def __init__(self, body):
        super().__init__()
        body.register_forward_pre_hook(self.pre_hook)
        body.register_forward_hook(self.forward_hook)
        self.shape = None
        
    def pre_hook(self, module, input):
        x = input[0]
        self.shape = x.shape
        return (x.view(-1, *x.shape[2:]),)
    
    def forward_hook(self, module, input, x):
        return x.view(*self.shape[:2], *x.shape[1:])

In [24]:
#export
def conv3(ni,nf,stride=1):
    return ConvLayer(ni, nf, (5,3,3), stride=(1,stride,stride), ndim=3, padding=(2,1,1))

In [25]:
#export
class Batchify(Module):
    def forward(self, x): return x.transpose(1,2)

class DeBatchify(Module):
    def forward(self, x):
        x_t = x.transpose(1,2)
        x_c = x_t.contiguous().view(-1, *x_t.shape[2:])
        return x_c

def get_3d_head(concat_pool=True):
    pool, feat = (AdaptiveConcatPool2d(1), 64*2) if concat_pool else (nn.AdaptiveAvgPool2d(1), 64)
    m = nn.Sequential(Batchify(),
        conv3(512,256,2), # 8
        conv3(256,128,2), # 4
        conv3(128, 64,2), # 2
        DeBatchify(), pool, Flatten(), nn.Linear(feat,6))
    init_cnn(m)
    return m

## Ignore Padding in Loss

In [26]:
#export
class DePadLoss(Callback):
    def __init__(self, pad_idx=-1): 
        super().__init__()
        store_attr(self, 'pad_idx')

    def after_pred(self):
        learn = self.learn
        targ = learn.yb[0].view(-1, *learn.yb[0].shape[2:])
        if targ.shape[0] != self.pred.shape[0]:
            pred = learn.pred.view(-1, *learn.pred.shape[2:])
        else: pred = learn.pred
        
        mask = targ[:,-1] != self.pad_idx
        
        learn.pred = pred[mask]
        learn.yb = (targ[mask],)

## Training Features - By Batch

In [27]:
dbunch_feat = get_3d_dbunch_feat(df_any, bs=16)

In [28]:
m = get_3d_head()
learn = get_learner(dbunch_feat, m)

In [29]:
learn.add_cb(DePadLoss())

In [30]:
do_fit(learn, 1, 1e-2)

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.108678,0.157843,0.946956,0.905628,02:45


In [31]:
learn.summary()

Sequential (Input shape: 16 x 48 x 512 x 8 x 8)
Layer (type)         Output Shape         Param #    Trainable 
Batchify             16 x 512 x 48 x 8 x 8 0          False     
________________________________________________________________
Conv3d               16 x 256 x 48 x 4 x 4 5,898,240  True      
________________________________________________________________
BatchNorm3d          16 x 256 x 48 x 4 x 4 512        True      
________________________________________________________________
ReLU                 16 x 256 x 48 x 4 x 4 0          False     
________________________________________________________________
Conv3d               16 x 128 x 48 x 2 x 2 1,474,560  True      
________________________________________________________________
BatchNorm3d          16 x 128 x 48 x 2 x 2 256        True      
________________________________________________________________
ReLU                 16 x 128 x 48 x 2 x 2 0          False     
___________________________________________

## Train on Slice

In [27]:
m = get_3d_head()

In [28]:
dbunch = get_3d_dbunch_aug(df_any, bs=10)
config=dict(custom_head=m, init=None)
learn = get_learner(dbunch, resnet18, get_loss(), config=config)

In [29]:
# learn.model[0] = ReshapeCNNBody(learn.model[0])

In [30]:
hook = ReshapeBodyHook(learn.model[0])

In [31]:
learn.add_cb(DePadLoss())

In [32]:
do_fit(learn, 1, 1e-2)

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.278352,0.272392,0.890677,0.803443,09:38


## Export

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_metadata.ipynb.
Converted 01_preprocess.ipynb.
Converted 01_preprocess_mean_std.ipynb.
Converted 02_train.ipynb.
Converted 03_train3d.ipynb.
Converted 03_train3d_01_train3d.ipynb.
Converted 03_train3d_01b_train_lstm.ipynb.
Converted 03_train3d_02_train_head.ipynb.
Converted 03_trainfull3d.ipynb.
Converted 04_trainSeq_01_lstm.ipynb.
Converted 04_trainSeq_02_transformer.ipynb.
Converted 04_trainSeq_03_lstm_seutao.ipynb.
Converted 05_train_adjacent.ipynb.
Converted 05_train_adjacent_01_5c.ipynb.
Converted 05_train_adjacent_02_3c.ipynb.
Converted 06_seutao_features.ipynb.
Converted 06_seutao_features_01_simple_lstm_20ep.ipynb.
Converted 06_seutao_features_01b_simple_lstm_10ep.ipynb.
Converted 06_seutao_features_01c_simple_lstm_meta.ipynb.
Converted 06_seutao_features_01d_simple_lstm_meta_fulldataset.ipynb.
Converted 06_seutao_features_02_2ndPlace.ipynb.
Converted 06_seutao_features_03_1stPlace.ipynb.
Converted 07_train_3d_lstm.ipynb.
